# Using plaidml to understand performance gains for a convolutional architecture

### 1. Import the libraries

In [1]:
import plaidml.keras
plaidml.keras.install_backend()

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

### 2. Adjusting the model config

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

### 3. Data Preparation

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### 4. Set the model architecture

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

INFO:plaidml:Opening device "metal_amd_radeon_pro_460.0"


### 5. Compile the model

In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 6. Train the CNN model

#### 6.1 Using `cpu`

In [6]:
%%time 
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 96s 2ms/step - loss: 0.2684 - acc: 0.9175 - val_loss: 0.0629 - val_acc: 0.9811
Epoch 2/12
60000/60000 [==============================] - 95s 2ms/step - loss: 0.0879 - acc: 0.9740 - val_loss: 0.0391 - val_acc: 0.9863
Epoch 3/12
60000/60000 [==============================] - 96s 2ms/step - loss: 0.0650 - acc: 0.9807 - val_loss: 0.0361 - val_acc: 0.9875
Epoch 4/12
60000/60000 [==============================] - 95s 2ms/step - loss: 0.0556 - acc: 0.9833 - val_loss: 0.0317 - val_acc: 0.9893
Epoch 5/12
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0474 - acc: 0.9856 - val_loss: 0.0292 - val_acc: 0.9895
Epoch 6/12
60000/60000 [==============================] - 96s 2ms/step - loss: 0.0419 - acc: 0.9869 - val_loss: 0.0278 - val_acc: 0.9906
Epoch 7/12
60000/60000 [==============================] - 98s 2ms/step - loss: 0.0371 - acc: 0.9887 - val_loss: 0.0264 - val_acc

#### 6.2 Using `metal_amd_gpu`

In [12]:
# using plaidml backend
# fit the model
%timeit
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 22s 358us/step - loss: 0.2738 - acc: 0.9159 - val_loss: 0.0567 - val_acc: 0.9814
Epoch 2/12
60000/60000 [==============================] - 18s 307us/step - loss: 0.0897 - acc: 0.9727 - val_loss: 0.0394 - val_acc: 0.9871
Epoch 3/12
60000/60000 [==============================] - 18s 306us/step - loss: 0.0659 - acc: 0.9801 - val_loss: 0.0380 - val_acc: 0.9870
Epoch 4/12
60000/60000 [==============================] - 18s 304us/step - loss: 0.0528 - acc: 0.9843 - val_loss: 0.0296 - val_acc: 0.9899
Epoch 5/12
60000/60000 [==============================] - 18s 307us/step - loss: 0.0465 - acc: 0.9859 - val_loss: 0.0286 - val_acc: 0.9897
Epoch 6/12
60000/60000 [==============================] - 18s 306us/step - loss: 0.0406 - acc: 0.9874 - val_loss: 0.0296 - val_acc: 0.9895
Epoch 7/12
60000/60000 [==============================] - 18s 307us/step - loss: 0.0367 - acc: 0.9892 - val_loss: 0.

### 7. Evaluate the model

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02810245600938797
Test accuracy: 0.9899
